<a href="https://colab.research.google.com/github/andryll/MGR-IC/blob/main/codes/MGR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
!git clone https://github.com/andryll/MGR-IC.git

Cloning into 'MGR-IC'...
remote: Enumerating objects: 1048, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 1048 (delta 15), reused 8 (delta 0), pack-reused 999
Receiving objects: 100% (1048/1048), 1.13 GiB | 29.86 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (1005/1005), done.


###Leitura das Músicas

In [120]:
def readSongs (genre, numSongs, sr=44100):

  genrelist = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
                'metal', 'pop', 'reggae', 'rock']

  if genre != 'all':
    genrelist = [genre]

  songs = []

  for g in genrelist:

    dir_path = os.path.join('/content/MGR-IC/songs/', g)

    files = os.listdir(dir_path)
    files.sort

    for i in range(numSongs):
      songs.append(librosa.load(os.path.join(dir_path, files[i]), sr=sr, mono = True))
      songs[-1] = songs[-1] + (g,)
 # print(songs)

  min_len = min(len(song[0]) for song in songs)

  min_songs = []
  for song in songs:
      if len(song[0]) > min_len:
        start_trim = int((len(song[0]) - min_len) / 2)
        end_trim = len(song[0]) - start_trim
        trimmed_audio = librosa.util.normalize(song[0][start_trim:end_trim])
        min_songs.append(trimmed_audio)
      else:
        min_songs.append(song[0])

  new_songs = []
  for i in range(len(songs)):
    tupla = (min_songs[i], songs[i][1], songs[i][2])
    new_songs.append(tupla)

  return new_songs


###Extração das Features

In [127]:
def featureExtraction (songs, feature, sr=44100, frame=512):

  zcrlist = []
  rmslist = []
  mfcclist = []
  sctoidlist = []
  srlofflist99 = []
  srlofflist1 = []

  for i in songs:

    match feature:
      case 'zcr':
        zcr = librosa.feature.zero_crossing_rate(i[0], frame_length = frame, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(zcr[0]).transpose()
        aux['Classe'] = i[2]
        zcrlist.append(aux)

      case 'rms':
        rms = librosa.feature.rms(y=i[0], frame_length = frame, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(rms[0]).transpose()
        aux['Classe'] = i[2]
        rmslist.append(aux)

      case 'mfcc':
        mfcc = librosa.feature.mfcc(y=i[0], sr=sr)
        aux = pd.DataFrame(mfcc).transpose()
        aux['Classe'] = i[2]
        mfcclist.append(aux)

      case 'centroid':
        sctoid = librosa.feature.spectral_centroid(y=i[0], sr=sr, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(sctoid[0]).transpose()
        aux['Classe'] = i[2]
        sctoidlist.append(aux)

      case 'rolloff':
        srloff99 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = round(frame/2), center = True, roll_percent=0.95)
        srloff1 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = round(frame/2), center = True, roll_percent=0.05)
        aux = pd.DataFrame(srloff99).transpose()
        aux['Classe'] = i[2]
        srlofflist99.append(aux)
        aux = pd.DataFrame(srloff1).transpose()
        aux['Classe'] = i[2]
        srlofflist1.append(aux)

      case 'all':
        zcr = librosa.feature.zero_crossing_rate(i[0], frame_length = frame, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(zcr[0]).transpose()
        aux['Classe'] = i[2]
        zcrlist.append(aux)

        rms = librosa.feature.rms(y=i[0], frame_length = frame, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(rms[0]).transpose()
        aux['Classe'] = i[2]
        rmslist.append(aux)

        mfcc = librosa.feature.mfcc(y=i[0], sr=sr)
        aux = pd.DataFrame(mfcc).transpose()
        aux['Classe'] = i[2]
        mfcclist.append(aux)

        sctoid = librosa.feature.spectral_centroid(y=i[0], sr=sr, hop_length = round(frame/2), center = True)
        aux = pd.DataFrame(sctoid[0]).transpose()
        aux['Classe'] = i[2]
        sctoidlist.append(aux)

        srloff99 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = round(frame/2), center = True, roll_percent=0.95)
        srloff1 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = round(frame/2), center = True, roll_percent=0.05)
        aux = pd.DataFrame(srloff99[0]).transpose()
        aux['Classe'] = i[2]
        srlofflist99.append(aux)
        aux = pd.DataFrame(srloff1[0]).transpose()
        aux['Classe'] = i[2]
        srlofflist1.append(aux)

      case _:
        print('Feature Inválida')

  match feature:
    case 'zcr':
      dfzcr = pd.concat(zcrlist, axis=0)
      return dfzcr

    case 'rms':
      dfrms = pd.concat(rmslist, axis=0)
      return dfrms

    case 'mfcc':
      dfmfcc = pd.concat(mfcclist, axis=0)
      return dfmfcc

    case 'centroid':
      dfcentroid = pd.concat(sctoidlist, axis=0)
      return dfcentroid

    case 'roloff':
      dfrolloff01 = pd.concat(sclofflist1, axis=0)
      dfrolloff99 = pd.concat(sclofflist99, axis=0)
      return [dfrolloff01, dfrolloff99]

    case 'all':
      dfzcr = pd.concat(zcrlist, axis=0)
      dfrms = pd.concat(rmslist, axis=0)
      dfmfcc = pd.concat(mfcclist, axis=0)
      dfcentroid = pd.concat(sctoidlist, axis=0)
      dfrolloff01 = pd.concat(sclofflist1, axis=0)
      dfrolloff99 = pd.concat(sclofflist99, axis=0)
      return dfzcr, dfrms, dfmfcc, dfcentroid, dfrolloff01, dfrolloff99

### Gráfico Cotovelo

In [137]:
def elbow(frame_lenghts, dflist):

  for df in dflist:
    X = df.iloc[:, :-1]
    Y = df.iloc[:, -1]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state = 14)

### Testes

In [126]:
songs = readSongs ('all', 10, 44100)
# [df, df2] = featureExtraction (songs, 'rolloff', 44100)
# df.head()
# df2.head()

In [125]:
dfzcr = featureExtraction(songs, 'zcr', 44100, frame=512)
display(dfzcr)

,0,1,2,3,4,5,6,7,8,9,...,5159,5160,5161,5162,5163,5164,5165,5166,5167,Classe
0,0.009766,0.017578,0.011719,0.007812,0.009766,0.011719,0.015625,0.015625,0.011719,0.009766,...,0.080078,0.087891,0.087891,0.082031,0.072266,0.041016,0.017578,0.013672,0.011719,blues
0,0.056641,0.082031,0.039062,0.042969,0.050781,0.031250,0.015625,0.046875,0.058594,0.037109,...,0.044922,0.033203,0.005859,0.025391,0.037109,0.029297,0.029297,0.017578,0.001953,blues
0,0.005859,0.029297,0.058594,0.058594,0.029297,0.027344,0.037109,0.029297,0.029297,0.035156,...,0.150391,0.117188,0.093750,0.107422,0.117188,0.099609,0.089844,0.095703,0.052734,blues
0,0.025391,0.058594,0.060547,0.048828,0.046875,0.058594,0.056641,0.046875,0.050781,0.054688,...,0.050781,0.046875,0.048828,0.039062,0.029297,0.027344,0.031250,0.031250,0.019531,blues
0,0.044922,0.080078,0.082031,0.089844,0.068359,0.056641,0.052734,0.048828,0.031250,0.031250,...,0.056641,0.060547,0.060547,0.042969,0.021484,0.017578,0.013672,0.017578,0.017578,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.009766,0.015625,0.015625,0.021484,0.017578,0.011719,0.013672,0.015625,0.017578,0.017578,...,0.044922,0.037109,0.039062,0.050781,0.052734,0.044922,0.046875,0.048828,0.033203,rock
0,0.076172,0.154297,0.148438,0.134766,0.121094,0.115234,0.099609,0.083984,0.066406,0.068359,...,0.076172,0.064453,0.095703,0.093750,0.060547,0.072266,0.068359,0.066406,0.060547,rock
0,0.029297,0.042969,0.041016,0.058594,0.089844,0.089844,0.070312,0.080078,0.119141,0.125000,...,0.083984,0.070312,0.058594,0.070312,0.076172,0.083984,0.080078,0.085938,0.070312,rock
0,0.025391,0.054688,0.066406,0.095703,0.091797,0.054688,0.039062,0.042969,0.048828,0.062500,...,0.039062,0.029297,0.046875,0.052734,0.041016,0.044922,0.052734,0.054688,0.033203,rock


In [128]:
frame_lenghts = [16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]

dflist = []
for f in frame_lenghts:
  dflist.append(featureExtraction(songs, 'zcr', 44100, frame=f))

dflist[3].head()

,0,1,2,3,4,5,6,7,8,9,...,20661,20662,20663,20664,20665,20666,20667,20668,20669,Classe
0,0.007812,0.023438,0.023438,0.015625,0.015625,0.015625,0.007812,0.015625,0.015625,0.007812,...,0.015625,0.015625,0.015625,0.015625,0.007812,0.007812,0.015625,0.031250,0.023438,blues
0,0.070312,0.164062,0.085938,0.062500,0.078125,0.062500,0.085938,0.039062,0.000000,0.023438,...,0.023438,0.015625,0.039062,0.031250,0.000000,0.000000,0.007812,0.007812,0.000000,blues
0,0.007812,0.007812,0.007812,0.015625,0.046875,0.070312,0.046875,0.023438,0.015625,0.070312,...,0.101562,0.054688,0.101562,0.125000,0.117188,0.078125,0.062500,0.062500,0.031250,blues
0,0.031250,0.062500,0.062500,0.039062,0.039062,0.062500,0.054688,0.070312,0.070312,0.070312,...,0.046875,0.054688,0.023438,0.023438,0.039062,0.031250,0.015625,0.031250,0.023438,blues
0,0.054688,0.078125,0.078125,0.101562,0.046875,0.015625,0.070312,0.125000,0.140625,0.085938,...,0.000000,0.031250,0.031250,0.015625,0.007812,0.023438,0.023438,0.039062,0.039062,blues


In [135]:
X = dflist[3].iloc[:, :-1]
Y = dflist[3].iloc[:, -1]

print(X.shape)
print(Y.shape)

(100, 20670)
(100,)
